In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_avccmdm422',
    proc_date,
    'temp_sap_avccmdm422'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm422 (
lifnr STRING,
land1 STRING, 
lfreg STRING,  
spras STRING, 
vtext STRING,   
proc_date DATE,
file_creation_ts TIMESTAMP
)
TBLPROPERTIES 
(
	DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE = TRUE,
	DELTA.AUTOOPTIMIZE.AUTOCOMPACT = TRUE
)
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm422
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm422
SELECT lifnr, land1, lfreg, spras, vtext, proc_date, file_creation_ts FROM temp_sap_avccmdm422
""")